In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import Project_One as AAC

###########################
# Data Manipulation / Model
###########################

# hard-wired database user and pass
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AAC.AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# logo img path
image_filename = './Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    dcc.Link(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 alt='Grazioso Logo', height="200", width="200"),
                href="snhu.edu"),
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.H1("Alec Novak - SNHU"),
    html.Hr(),
# radio items that pass filter value to callback
    dcc.RadioItems(
        id="animal_filter",
        options=[{'label': i, 'value': i} for i in ["All Animals", "Water", "Wilderness/Mountains", "Disaster/Tracking"]],
value="All Animals"),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
# interactivity to make data more manageable
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
# filtering callback
@app.callback(
    Output('datatable-id', 'data'),
    Input('animal_filter', 'value')
)

def animal_filter(filter):
    # load data
    df = pd.DataFrame.from_records(db.read({}))
    
    # query data dependent on filter
    #reset filter
    if (filter == "All Animals"):
        df = pd.DataFrame.from_records(db.read({}))
        
    # filter animals that meet requirements for water rescue
    elif (filter == 'Water'):
        df = pd.DataFrame.from_records(db.read({"breed" : {
            "$in":
                ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                'sex_upon_outcome': 'Intact Female',
                'age_upon_outcome_in_weeks': {'$gte': 26},
                'age_upon_outcome_in_weeks': {'$lte': 156}}))
        
    # filter animals that meet requirements for wilderness and mountain rescue 
    elif (filter == 'Wilderness/Mountains'):
        df = pd.DataFrame.from_records(db.read({"breed" : {
            "$in":
                ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte': 26},
                'age_upon_outcome_in_weeks': {'$lte': 156}}))
        
    #filter animals that meet requirements for disaster and tracking rescue
    elif (filter == 'Disaster/Tracking'):
        df = pd.DataFrame.from_records(db.read({"breed" : {
            "$in":
                ["Doberman Pinscher", "Alaskan Malamute", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks': {'$gte': 20},
                'age_upon_outcome_in_weeks': {'$lte': 300}}))
        
    # if a different value than expected, reset filter
    else:
        df = pd.DataFrame.from_records(db.read({}))
        
    # drop _id column
    df.drop(columns=['_id'],inplace=True)
    data = df.to_dict('records')
    
    return (data)

# Display the breeds of animal based on quantity represented in
# the data table, connected to filter
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', 'derived_viewport_data'),
    Input('animal_filter', 'value')])
def update_graphs(viewData, filter):
    
    df = pd.DataFrame.from_dict(viewData)
    
    # query data dependent on filter
    # reset filter when "All Animals" is passed
    if (filter == "All Animals"):
        df = pd.DataFrame.from_records(db.read({}))
    
    # configure the graph with the resulting dataframe
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


{'_id': ObjectId('679fb19627a977cd0fdcb975'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': 'Transfer',
 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('679fb1a90ac3e65c163ae588'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_

Dash app running on http://127.0.0.1:13925/
{'_id': ObjectId('679fb19627a977cd0fdcb975'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': 'Transfer',
 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('679fb1a90ac3e65c163ae588'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name'

 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('67a2859752f2dc82599ad54e'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 'rec_num': 22,
 'sex_upon_outcome'156.767857142857,
 'animal_id': 'B746874',
 'animal_type': : 'Neutered Male'}
{'_id': ObjectId('67b762048c9814ab03039bdf'),
 'age_upon_outcome': '1 year',
 'age_upon_outcome_in_weeks': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime'52.4821428571429,
 'animal_id': 'A743231',
 'animal_type': 'Cat',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black',
 'date_of_birth': '2016-02-06',
 'datetime': '2017-02-07 09:00:00',
 'location_lat': 30.2920163966506,
 : '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear''location_long': -97.4266291874198,
 'monthyear': '2017-02-07T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': '2017-04-11T09:00:00',
 'name': '',
 'outc

 'location_long': -97.3924668221853'animal_type': 'Dog',
 'breed': 'Boxer/Bullmastiff',
 'color',
 ,
 'monthyear': 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': : 'Brown Brindle/White',
 'date_of_birth': '2015-01-18',
 'datetime': '2017-02-11 12:30:00',
 'location_lat': 30.4551148649096,
 'location_long': -97.3087780473978,
 '2015-06-27T13:19:00',
 'name': '',
 'outcome_subtype': '''monthyear': '2017-02-11T12:30:00',
 'name': '*Kawhi',
 ,
 'outcome_type': 'Adoption',
 'rec_num': 347,
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('67b76c17bb1e4e8855dad054'),
 'age_upon_outcome': '2 months',
 'outcome_subtype': '',
 'outcome_type': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': 'Transfer',
 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('67a27c2509553cecbe235195'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type''

 'color': 'White/Black'
'age_upon_outcome_in_weeks': 104.393948412698,
 'animal_id': ,
 'date_of_birth': '2017-09-09',
 {'_id': ObjectId('67b762048c9814ab03039bd8'),
 'age_upon_outcome': '3 months',
 'age_upon_outcome_in_weeks': 'datetime': '2017-11-26 11:37:00',
 'location_lat': 30.3491408570609,
 'location_long': -97.4793291291553,
 'monthyear': '2017-11-26T11:37:00',
 'name': 'A730354'14.0890873015873,
 'animal_id': 'A664290',
 'animal_type': 'Cat',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Tortie',
 ,
 'animal_type': 'Dog',
 'breed': 'Doberman Pinscher',
 'color': 'Blue/White',
 'date_of_birth': '2015-07-02',
 'datetime': '2017-07-01 18:11:00',
 'location_lat': 30.4815689674138,
 'location_long': -97.3694960927915,
 'date_of_birth': '2013-09-01',
 'datetime': '2013-12-08 14:58:00',
 'location_lat': 30.7583105481048,
 'location_long': -97.618292198845,
 'monthyear': '2013-12-08T14:58:00',
 'name': '*Taylor',
 'outcome_subtype': '',
 'outcome_type': 'Adoption''monthyear': '2017-

...

 'outcome_type': 'Euthanasia',
 'rec_num': 334,
 'sex_upon_outcome': 'Unknown'}
{'_id': ObjectId('67b76c17bb1e4e8855dad04d'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.820833333333,
 'animal_id': 'A678393',
 'animal_type': 'Dog',
 'breed': 'Golden Retriever',
 'color': 'Brown',
 'date_of_birth': '2011-05-08',
 'datetime': '2014-05-09 17:54:00',
 'location_lat': 30.429299327634,
 'location_long': -97.6161732279484,
 'monthyear': '2014-05-09T17:54:00',
 'name': 'Bebe',
 'outcome_subtype': '',
 'outcome_type': 'Return to Owner',
 'rec_num': 315,
 'sex_upon_outcome': 'Spayed Female'}
{'_id': ObjectId('67b76c17bb1e4e8855dad04e'),
 'age_upon_outcome': '2 months',
 'age_upon_outcome_in_weeks': 12.959623015873,
 'animal_id': 'A746862',
 'animal_type': 'Dog',
 'breed': 'Doberman Pinscher',
 'color': 'Black/Brown',
 'date_of_birth': '2017-01-14',
 'datetime': '2017-04-14 17:13:00',
 'location_lat': 30.4105077089376,
 'location_long': -97.6502521344198,
 'monthyear': '2017

{'_id': ObjectId('679fb19627a977cd0fdcb975'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': 'Transfer',
 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('679fb1a90ac3e65c163ae588'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_

{'_id': ObjectId('679fb19627a977cd0fdcb975'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': 'Transfer',
 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('679fb1a90ac3e65c163ae588'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_

{'_id': ObjectId('679fb19627a977cd0fdcb975'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': 'Transfer',
 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('679fb1a90ac3e65c163ae588'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_

{'_id': ObjectId('679fb19627a977cd0fdcb975'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_type': 'Transfer',
 'rec_num': '1',
 'sex_upon_outcome': 'Neutered Male'}
{'_id': ObjectId('679fb1a90ac3e65c163ae588'),
 'age_upon_outcome': '3 years',
 'age_upon_outcome_in_weeks': 156.767857142857,
 'animal_id': 'B746874',
 'animal_type': 'Dog',
 'breed': 'Domestic Shorthair Mix',
 'color': 'Black/White',
 'date_of_birth': '2014-04-10',
 'datetime': '2017-04-11 09:00:00',
 'location_lat': 30.5066578739455,
 'location_long': -97.3408780722188,
 'monthyear': '2017-04-11T09:00:00',
 'name': '',
 'outcome_subtype': 'SCRP',
 'outcome_